In [ ]:
# Save the fine-tuned model
model_save_path = "./fine_tuned_sentiment_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model saved to: {model_save_path}")

# Test the fine-tuned model
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
    confidence = torch.max(predictions).item()
    predicted_class = torch.argmax(predictions).item()
    sentiment = "positive" if predicted_class == 1 else "negative"
    
    return sentiment, confidence

# Test on new examples
test_texts = [
    "This movie was absolutely incredible!",
    "I hate this product, it's terrible.",
    "The service was okay, nothing special.",
    "Best purchase I've ever made!",
    "Complete waste of time and money."
]

print("\nTesting fine-tuned model:")
print("=" * 60)

for text in test_texts:
    sentiment, confidence = predict_sentiment(text)
    print(f"Text: {text}")
    print(f"Prediction: {sentiment} (confidence: {confidence:.3f})")
    print("-" * 40)

# Load model from saved checkpoint (demonstration)
print("\nDemonstrating model loading from checkpoint:")
from transformers import pipeline

# Create pipeline from saved model
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model_save_path,
    tokenizer=model_save_path
)

# Quick test
result = sentiment_pipeline("This fine-tuning worked great!")
print(f"Pipeline result: {result}")

print("\n🎉 Fine-tuning completed successfully!")
print("Model ready for deployment and inference!")

## Example 5: Model Testing and Deployment

In [ ]:
# Start fine-tuning
print("Starting fine-tuning...")
train_result = trainer.train()

# Training results
print("\nTraining completed!")
print(f"Final training loss: {train_result.training_loss:.4f}")

# Evaluate on validation set
eval_results = trainer.evaluate()
print(f"\nValidation Results:")
for key, value in eval_results.items():
    if key.startswith('eval_'):
        metric_name = key.replace('eval_', '')
        print(f"{metric_name.capitalize()}: {value:.4f}")

# Training history (if available)
if hasattr(trainer.state, 'log_history'):
    print(f"\nTraining steps completed: {len(trainer.state.log_history)}")
    
    # Show loss progression
    losses = [log['train_loss'] for log in trainer.state.log_history if 'train_loss' in log]
    if losses:
        print(f"Loss progression: {losses[0]:.4f} -> {losses[-1]:.4f}")
        print(f"Loss improvement: {((losses[0] - losses[-1]) / losses[0] * 100):.2f}%")

## Example 4: Running the Fine-tuning Process

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define metrics computation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("Training configuration:")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Learning rate: {training_args.learning_rate}")
print(f"Weight decay: {training_args.weight_decay}")

## Example 3: Training Configuration and Metrics

In [ ]:
# Load pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2  # binary classification
)

print(f"Model: {model_name}")
print(f"Number of parameters: {model.num_parameters():,}")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['text'], 
        truncation=True, 
        padding=True,
        max_length=128
    )

# Apply tokenization to datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print("\nTokenized example:")
print(f"Input IDs shape: {train_dataset[0]['input_ids'].shape}")
print(f"Attention mask shape: {train_dataset[0]['attention_mask'].shape}")
print(f"Label: {train_dataset[0]['label']}")

## Example 2: Model and Tokenizer Setup

In [ ]:
# Sample data for sentiment analysis fine-tuning
sample_data = {
    'text': [
        "This product is amazing! I love it so much.",
        "Terrible experience, would not recommend.",
        "It's okay, nothing special but works fine.",
        "Outstanding quality and great customer service!",
        "Worst purchase ever, complete waste of money.",
        "Pretty good value for the price point.",
        "Absolutely fantastic! Exceeded my expectations.",
        "Not bad, but could be better quality.",
        "Love this! Will definitely buy again.",
        "Disappointing results, expected much more."
    ],
    'label': [1, 0, 1, 1, 0, 1, 1, 0, 1, 0]  # 1: positive, 0: negative
}

# Create dataset
dataset = Dataset.from_dict(sample_data)

# Split into train/validation
train_size = int(0.8 * len(dataset))
train_dataset = dataset.select(range(train_size))
val_dataset = dataset.select(range(train_size, len(dataset)))

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"\nSample data:")
print(train_dataset[0])

# Fine-tuning Basics with Hugging Face 🎯

Fine-tuning adapts pre-trained models to your specific tasks and datasets. It's the key to getting state-of-the-art performance on domain-specific problems.

## What is Fine-tuning?

**Fine-tuning** customizes pre-trained models:
- **Transfer Learning**: Leverage pre-trained knowledge
- **Task Adaptation**: Specialize for your specific use case
- **Examples**: Custom classification, domain-specific NER, specialized generation

## Learning Objectives

By the end of this notebook, you'll know how to:
1. Prepare datasets for fine-tuning
2. Set up training configurations
3. Monitor training progress and metrics
4. Handle overfitting and optimization issues
5. Save and load fine-tuned models
6. Best practices for different model types

Let's fine-tune! ⚡

In [ ]:
# Import essential libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
import torch
print('Fine-tuning basics notebook ready!')

## Example 1: Data Preparation for Fine-tuning